## Importing Libraries

In [1]:
import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import random
words=[]
classes = []
documents = []
ignore_words = ['?', '!']

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Kunal\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## Reading the data file

In [2]:
data_file = open('GL Bot.json').read()
intents = json.loads(data_file)

In [3]:
intents

{'intents': [{'tag': 'Intro',
   'patterns': ['hi',
    'how are you',
    'is anyone there',
    'hello',
    'whats up',
    'hey',
    'yo',
    'listen',
    'please help me',
    'i am learner from',
    'i belong to',
    'aiml batch',
    'aifl batch',
    'i am from',
    'my pm is',
    'blended',
    'online',
    'i am from',
    'hey ya',
    'talking to you for first time'],
   'responses': ['Hello! how can i help you ?'],
   'context_set': ''},
  {'tag': 'Exit',
   'patterns': ['thank you',
    'thanks',
    'cya',
    'see you',
    'later',
    'see you later',
    'goodbye',
    'i am leaving',
    'have a Good day',
    'you helped me',
    'thanks a lot',
    'thanks a ton',
    'you are the best',
    'great help',
    'too good',
    'bye',
    'exit',
    'you are a good learning buddy'],
   'responses': ['I hope I was able to assist you, Good Bye'],
   'context_set': ''},
  {'tag': 'Olympus',
   'patterns': ['olympus',
    'explain me how olympus works',
    'I a

## Tokenization

In [4]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        
        #tokenize each word
        
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        
        #add documents in the corpus
        
        documents.append((w, intent['tag']))

        # add to our classes list
        
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

## Lemmatization, Lower Casing, Removing Duplicates

In [5]:
# lemmatize, lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# sort classes
classes = sorted(list(set(classes)))

# documents = combination between patterns and intents
print (len(documents), "documents")

# classes = intents
print (len(classes), "classes", classes)

# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)

pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

139 documents
9 classes ['Bot', 'Exit', 'Intro', 'NN', 'Olympus', 'Profane', 'SL', 'Ticket', 'anymore']
169 unique lemmatized words ['a', 'able', 'access', 'activation', 'ada', 'adam', 'aifl', 'aiml', 'am', 'an', 'ann', 'answer', 'anyone', 'are', 'artificial', 'backward', 'bad', 'bagging', 'batch', 'bayes', 'belong', 'best', 'blended', 'bloody', 'boosting', 'bot', 'buddy', 'bye', 'classification', 'contact', 'cool', 'create', 'cross', 'cya', 'day', 'deep', 'did', 'diffult', 'do', 'ensemble', 'epoch', 'exit', 'explain', 'fine', 'first', 'for', 'forest', 'forward', 'from', 'function', 'good', 'goodbye', 'got', 'gradient', 'great', 'hate', 'have', 'hell', 'hello', 'help', 'helped', 'hey', 'hi', 'hidden', 'hmm', 'hmmmm', 'hour', 'how', 'hyper', 'i', 'imputer', 'in', 'intelligence', 'is', 'it', 'jerk', 'joke', 'knn', 'later', 'layer', 'learner', 'learning', 'leaving', 'link', 'listen', 'logistic', 'lot', 'machine', 'me', 'ml', 'my', 'naive', 'name', 'nb', 'net', 'network', 'neural', 'no', '

## Training Data

In [6]:
# create our training data
training = []

# create an empty array for our output
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:

    # initialize our bag of words
    bag = []
    
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

    # shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)

# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])

C:\Users\Kunal\Anaconda3\lib\site-packages\ipykernel_launcher.py:31: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


## Model Creation 

In [7]:
model = Sequential()
model.add(Dense(512, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=100, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

Epoch 1/100
28/28 [==============================] - 0s 3ms/step - loss: 2.2253 - accuracy: 0.1135
Epoch 2/100
28/28 [==============================] - 0s 3ms/step - loss: 1.9825 - accuracy: 0.2797
Epoch 3/100
28/28 [==============================] - 0s 5ms/step - loss: 1.8781 - accuracy: 0.2857
Epoch 4/100
28/28 [==============================] - 0s 8ms/step - loss: 1.7953 - accuracy: 0.3908
Epoch 5/100
28/28 [==============================] - 0s 4ms/step - loss: 1.6103 - accuracy: 0.4845
Epoch 6/100
28/28 [==============================] - 0s 5ms/step - loss: 1.4241 - accuracy: 0.6008
Epoch 7/100
28/28 [==============================] - 0s 9ms/step - loss: 1.2605 - accuracy: 0.6242
Epoch 8/100
28/28 [==============================] - 0s 7ms/step - loss: 1.1001 - accuracy: 0.6216
Epoch 9/100
28/28 [==============================] - 0s 8ms/step - loss: 0.8611 - accuracy: 0.7105
Epoch 10/100
28/28 [==============================] - 0s 7ms/step - loss: 0.8111 - accuracy: 0.7885
Epoch 11/

28/28 [==============================] - 0s 7ms/step - loss: 0.0043 - accuracy: 1.0000
Epoch 83/100
28/28 [==============================] - 0s 7ms/step - loss: 0.0035 - accuracy: 1.0000
Epoch 84/100
28/28 [==============================] - 0s 6ms/step - loss: 0.0039 - accuracy: 1.0000
Epoch 85/100
28/28 [==============================] - 0s 6ms/step - loss: 0.0055 - accuracy: 1.0000
Epoch 86/100
28/28 [==============================] - 0s 4ms/step - loss: 0.0034 - accuracy: 1.0000
Epoch 87/100
28/28 [==============================] - 0s 5ms/step - loss: 0.0033 - accuracy: 1.0000
Epoch 88/100
28/28 [==============================] - 0s 5ms/step - loss: 0.0031 - accuracy: 1.0000
Epoch 89/100
28/28 [==============================] - 0s 4ms/step - loss: 0.0020 - accuracy: 1.0000
Epoch 90/100
28/28 [==============================] - 0s 5ms/step - loss: 0.0032 - accuracy: 1.0000
Epoch 91/100
28/28 [==============================] - 0s 4ms/step - loss: 0.0025 - accuracy: 1.0000
Epoch 92/100


## Loading pickle files

In [8]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import pickle
import numpy as np
from tensorflow.keras.models import load_model
model = load_model('chatbot_model.h5')
import json
import random
intents = json.loads(open('GL Bot.json').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

## Chatbot functions

In [9]:
def clean_up_sentence(sentence):
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word - create short form for word
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words
# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence

def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words) 
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

def chatbot_response(text):
    ints = predict_class(text, model)
    res = getResponse(ints, intents)
    return res

## GUI with Tinker

##### Running the below code will launch a GUI

In [10]:
import tkinter
from tkinter import *


def send():
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)

    if msg != '':
        ChatLog.config(state=NORMAL)
        ChatLog.insert(END, "You: " + msg + '\n\n')
        ChatLog.config(foreground="#442265", font=("Verdana", 11 ))

        res = chatbot_response(msg)
        ChatLog.insert(END, "GL: " + res + '\n\n')

        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)

base = Tk()
base.title("Great Learning")
base.geometry("400x460")
base.resizable(width=FALSE, height=FALSE)

#Create Chat window
ChatLog = Text(base, bd=0, bg="white", height="8", width="50", font="Arial",)

ChatLog.config(state=DISABLED)

#Bind scrollbar to Chat window
scrollbar = Scrollbar(base, command=ChatLog.yview)
ChatLog['yscrollcommand'] = scrollbar.set

#Create Button to send message
SendButton = Button(base, font=("Verdana",11,'bold'), text=" Send", width="14", height=2,
                    bd=0, bg="#1974d2", activebackground="#0e39a9",fg='#ffffff',
                    command= send )

#Create the box to enter message
EntryBox = Text(base, bd=0, bg="white",width="32", height="2", font="Arial")
#EntryBox.bind("<Return>", send)


#Place all components on the screen
scrollbar.place(x=376,y=6, height=386)
ChatLog.place(x=6,y=6, height=386, width=370)
EntryBox.place(x=6, y=401, height=50, width=270)
SendButton.place(x=260, y=401, height=50)

base.mainloop()